In [1]:
from pycram.worlds.bullet_world import BulletWorld
from pycram.datastructures.pose import Pose
from pycram.datastructures.enums import ObjectType, WorldMode

world = BulletWorld(mode=WorldMode.GUI)

(robot-description) Could not get robot name from parameter server. Try again.
Failed to import Giskard messages, the real robot will not be available
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


ROSInitException: time is not initialized. Have you called init_node()?

In [2]:
import rospy

# Bullet World
This Notebook will show you the basics of working with the PyCRAM BulletWorld.

First we need to import and create a BulletWorld.

This new window is the BulletWorld, PyCRAMs internal physics simulation. You can use the mouse to move the camera around:

  * Press the left mouse button to rotate the camera
  * Press the right mouse button to move the camera 
  * Press the middle mouse button (scroll wheel) and move the mouse up or down to zoom
    
At the moment the BulletWorld only contains a floor, this is spawned by default when creating the BulletWorld. Furthermore, the gravity is set to 9.8 m^2, which is the same gravitation as the one on earth. 
    

To close the BulletWorld again please use the ```exit``` method since it will also terminate threads running in the background

In [20]:
world.exit()

To spawn new things in the BulletWorld we need to import the Object class and create and instance of it. 

In [5]:
from pycram.worlds.bullet_world import Object

milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([0, 0, 1]))

Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']


As you can see this spawns a milk floating in the air. What we did here was create a new Object which has the name "milk" as well as the type ```ObjectType.MILK ``` , is spawned from the file "milk.stl" and is at the position [0, 0, 1]. 

The type of an Object can either be from the enum ObjectType or a string. However, it is recommended to use the enum since this would make for a more consistent naming of types which makes it easier to work with types. But since the types of the enum might not fit your case you can also use strings. 

The first three of these parameters are required while the position is optional. As you can see it was sufficient to only specify the filename for PyCRAM to spawn the milk mesh. When only providing a filename, PyCRAM will search in its resource directory for a matching file and use it. 

For a complete list of all parameters that can be used to crate an Object please check the documentation. 



Since the Object is spawned, we can now interact with it. First we want to move it around and change its orientation

In [6]:
milk.set_position(Pose([1, 1, 1]))

In [7]:
milk.set_orientation(Pose(orientation=[1,0, 0, 1]))

In [8]:
milk.set_pose(Pose([0, 0, 1], [0, 0, 0, 1]))

In the same sense as setting the position or orientation, you can also get the position and orientation.

In [9]:
print(f"Position: \n{milk.get_position()}")

print(f"Orientation: \n{milk.get_orientation()}")

print(f"Pose: \n{milk.get_pose()}")

Position: 
x: 0.0
y: 0.0
z: 1.0
Orientation: 
x: 0.0
y: 0.0
z: 0.0
w: 1.0
Pose: 
header: 
  seq: 0
  stamp: 
    secs: 1729682205
    nsecs: 326739549
  frame_id: "map"
pose: 
  position: 
    x: 0.0
    y: 0.0
    z: 1.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0


## Attachments
You can attach Objects to each other simply by calling the attach method on one of them and providing the other as parameter. Since attachments are bi-directional it doesn't matter on which Object you call the method. 

First we need another Object

In [10]:
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1, 0, 1]))

Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']


In [11]:
milk.attach(cereal)

Now since they are attached to each other, if we move one of them the other will move in conjunction.

In [12]:
milk.set_position(Pose([1,1,1]))

In the same way the Object can also be detached, just call the detach method on one of the two attached Objects.

In [13]:
cereal.detach(milk)

## Links and Joints
Objects spawned from mesh files do not have links or joints, but if you spawn things from a URDF like a robot they will have a lot of links and joints. Every Object has two dictionaries as attributes, namely ```links``` and ```joints``` which contain every link or joint as key and a unique id, used by PyBullet, as value. 

We will see this at the example of the PR2:

In [14]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
print(pr2.links)

{'base_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25c8fb65b0>, 'base_bellow_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b11d6730>, 'base_laser_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b11d6f40>, 'fl_caster_rotation_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b11d6550>, 'fl_caster_l_wheel_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b16391f0>, 'fl_caster_r_wheel_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b16390d0>, 'fr_caster_rotation_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b16392b0>, 'fr_caster_l_wheel_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b1639370>, 'fr_caster_r_wheel_link': <pycram.object_descriptors.urdf.ObjectDescription.Link object at 0x7f25b1639400>, 'bl_caster_rotation_link': <pycram.object_descripto

For links there are similar methods available as for the pose. However, you can only **get** the position and orientation of a link. 

In [15]:
print(f"Position: \n{pr2.get_link_position('torso_lift_link')}")

print(f"Orientation: \n{pr2.get_link_orientation('torso_lift_link')}")

print(f"Pose: \n{pr2.get_link_pose('torso_lift_link')}")

Position: 
x: -0.05000000447034836
y: 0.0
z: 0.7906750440597534
Orientation: 
x: 0.0
y: 0.0
z: 0.0
w: 1.0
Pose: 
header: 
  seq: 0
  stamp: 
    secs: 1729682211
    nsecs: 922111749
  frame_id: "map"
pose: 
  position: 
    x: -0.05000000447034836
    y: 0.0
    z: 0.7906750440597534
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0


Methods available for joints are:

  * ```get_joint_state```
  * ```set_joint_state```
  * ```get_joint_limits```
  
We will see how these methods work at the example of the torso_lift_joint:

In [16]:
print(f"Joint limits: {pr2.get_joint_limits('torso_lift_joint')}")

print(f"Current Joint state: {pr2.get_joint_position('torso_lift_joint')}")

pr2.set_joint_position("torso_lift_joint", 0.2)

print(f"New Joint state: {pr2.get_joint_position('torso_lift_joint')}")

Joint limits: (0.0, 0.33)
Current Joint state: 0.0
New Joint state: 0.2


## Misc Methods
There are a few methods that don't fit any category but could be helpful anyway. The first two are ```get_color``` and ```set_color```, as the name implies they can be used to get or set the color for specific links or the whole Object. 

In [17]:
print(f"Pr2 forearm color: {pr2.get_link_color('r_forearm_link')}")

Pr2 forearm color: Color(R=0.699999988079071, G=0.699999988079071, B=0.699999988079071, A=1.0)


In [18]:
pr2.set_link_color("r_forearm_link", [1, 0, 0])

Lastly, there is ```get_AABB```, AABB stands for *A*xis *A*ligned *B*ounding *B*ox. This method returns two points in world coordinates which span a rectangle representing the AABB.

In [19]:
pr2.get_axis_aligned_bounding_box()

AxisAlignedBoundingBox(min_x=-0.0015000000000000005, min_y=-0.0015000000000000005, min_z=0.06949999999999999, max_x=0.0015000000000000005, max_y=0.0015000000000000005, max_z=0.0725)